# LSTM Time Series Deep Learning and Adversarial Attacks

## 1. About this Notebook

See the project [README](https://github.com/duanegoodner/lstm_adversarial_attack) for general information on the dataset and approach used in this notebook.

The implementation details for this project are encapsulated in various classes and methods defined in modules under the project `src` directory, and various intermediate data structures and logs are saved in the project `data` directory. Most of the code in this notebook simply instantiates top-level classes and makes calls to their methods without revealing implementation details. Please look to code in the `src` and `data` directories if you interested in lower level details. The import paths as well as the terminal output shown in this notebook will provide some guidance on where to look within those directories.

## 2. Project Setup

Before running this notebook, review the project README at https://github.com/duanegoodner/lstm_adversarial_attack, and complete all steps in the "How to run this project" section.

## 3. Imports
Most of the necessary standard library imports and external package imports are handled modules in the `src` directory, but we need to import a few here.

### 3.1 Standard Library Imports

In [ ]:
import pprint
import sys

### 3.2 External Packages

In [ ]:
import numpy as np
import torch

### 3.3 Internal Project Modules and Sub-packages
To help gain a sense of project structure, we will import internal packages and modules as-needed (i.e. immediately before the notebook code cells where they are first used). For now, we import the project `src` path defined in `lstm_adversarial_attack/notebooks/src_path`, add it to sys.path (so we can easily import project code), and we import project config files.

In [ ]:
import src_paths
sys.path.append(str(src_paths.lstm_adversarial_attack_pkg))
import lstm_adversarial_attack.config_paths as cfg_paths
import lstm_adversarial_attack.config_settings as cfg_set

## 3. Database Queries
We need to run four queries on the MIMIC-III PostgreSQL database. The paths to files containing the queries are stored in a list as `DB_QUERIES` in the project `config_paths` file:

In [ ]:
pprint.pprint(cfg_paths.DB_QUERIES)

To connect to the database, and execute the queries, we instantiate a `MimiciiiDatabaseAccess` object from module `mimiciii_database` of project sub-package `query_db` and use its .connect(), .run_sql_queries() and .close_connection() methods.

In [ ]:
import lstm_adversarial_attack.query_db.mimiciii_database as mdb

db_access = mdb.MimiciiiDatabaseAccess(
    dotenv_path=cfg_paths.DB_DOTENV_PATH, output_dir=cfg_paths.DB_OUTPUT_DIR
)
db_access.connect()
db_query_results = db_access.run_sql_queries(
    sql_query_paths=cfg_paths.DB_QUERIES
)
db_access.close_connection()

The results of each `.sql` query is saved to a `.csv` file. The path to each of these files is shown in the terminal output above. The output path of the queries is defined by variable `DB_OUTPUT_DIR` in the project `config_settings` file.

## 4. Preprocessor

### 4.1 Instantiate a Preprocessor object
We import the `preprocessor` module from internal sub-package `preprocess`, instantiate a `Preprocessor` object, and examine its .preprocess_modules data member. 

In [ ]:
import lstm_adversarial_attack.preprocess.preprocessor as pre
preprocessor = pre.Preprocessor()

We can get a general idea of how the `lstm_adversarial_attack.preprocess` sub-package works by looking at the `Preprocessor` object's `.preprocessor_modules` data member.

In [ ]:
pprint.pprint([item.__class__ for item in preprocessor.preprocess_modules])

* Prefilter reads the database query outputs into Pandas Dataframes, removes all data related to patients younger than 18 years in age, ensures consistent column naming formats, and takes care of datatype details.
* ICUStayMeasurementCombiner performs various joins (aka "merges" in the language of Pandas) to combine lab and vital sign measurement data with ICU stay data.
* FullAdmissionListBuilder generates a list consisting of one FullAdmissionData object per ICU stay. The attributes of a FullAdmissionData object include ICU stay info, and a dataframe containing the measurement and timestamp data for all vital sign and lab data associated with the ICU stay.
* FeatureBuilder resamples the time series datafame to one-hour intervals, imputes missing data, winsorizes measurement values (with cutoffs at the 5th and 95th global percentiles), and normalizes the measuremnt values so all data are between 0 and 1.
* FeatureFinalizer selects the data observation time window (default starts at hospital admission time and ends 48 hours after admission). This module outputs the entire dataset features as a list of numpy arrays, and the mortality labels as a list of integers. These data structures (saved as .pickle files) will be convenient starting points when the `tune_train` and `attack` sub-packages need to create PyTorch Datasets.

Now that we have a some background info, we are ready to run the Preprocessor.

In [ ]:
preprocessed_resources = preprocessor.preprocess()

## 5. Pytorch Dataset object

### 5.1 Create the Dataset
We import module `x19_mort_general_dataset` and use it along with files saved by the Preprocessor's Feature Finalizer module to insantiate a Pytorch Dataset

In [ ]:
import lstm_adversarial_attack.x19_mort_general_dataset as xmd
dataset = xmd.X19MGeneralDataset.from_feature_finalizer_output()

### 5.2 Examine the Dataset

#### 5.2.1 Dataset size, tensor shapes, and data types 

In [ ]:
print(f"Number of samples in dataset = {len(dataset)}\n")
print(f"Type returned by dataset.__getitem__ = {type(dataset[0])}\n")
print(
    f"Length of each tuple returned by dataset.__getitem__ = {len(dataset[0])}"
)
print(
    "\nObject type, dimensionality, and datatype of each element in a tuple"
    " returned by dataset.__getitem__:"
)
print(tuple([(type(item), item.dim(), item.dtype) for item in dataset[0]]))
print(f"\ninput size (# columns) of each feature matrix is:\n"
     f"{np.unique([item.shape[1] for item in dataset[:][0]]).item()}\n")

#### 5.2.2 Distributions of feature sequence lengths and label values

In [ ]:
print("Distribution of input sequence lengths (# rows):")
print("length\tcounts")
unique_sequence_lengths, sequence_length_counts = np.unique(
    [item.shape[0] for item in dataset[:][0]], return_counts=True
)
print(
    np.concatenate(
        (
            unique_sequence_lengths.reshape(-1, 1),
            sequence_length_counts.reshape(-1, 1),
        ),
        axis=1,
    )
)

print("\nLabel counts:")
print("value\tcounts")
unique_labels, label_counts = np.unique([dataset[:][1]], return_counts=True)
print(
    np.concatenate(
        (unique_labels.reshape(-1, 1), label_counts.reshape(-1, 1)), axis=1
    )
)

## 6. Model Hyperparameter Tuning

### 6.1 Tuning Parameters

We define the following architecture for our LSTM predictive model

| Item  | Detail #1 | Detail #2| Detail #3 |
| :-------------- | :---------| :---------| :---------
| Bidirectional LSTM | input size = 19 | # hidden states per direction = *x<sub>1</sub>* | activation = ReLU or Tanh  |
| Dropout | P<sub>dropout</sub> = *x<sub>2</sub>* | -  | -  |
| Fully Connected Layer #1 | input size = *2x<sub>1</sub>*| output size = *x<sub>3</sub>* | activation = ReLU or Tanh |
| Fully Connected Layer #2 | input size = *x<sub>3</sub>* | output size = 2 | activation = Softmax |
| Optimizer | type = SGD or RMSProp or Adam | learning rate = *x<sub>4</sub>* | - |

*x<sub>1</sub>*, *x<sub>2</sub>*, *x<sub>3</sub>*, *x<sub>4</sub>*, activation functions for the LSTM and FC #1, the optimizer type, and learning rate will be determined through hyperparameter tuning.

The `HyperParameterTuner` class in the `lstm_adversarial_attack.tune_train` sub-package implements a K-fold (default K = 5) cross-validation tuning scheme that utilizes the Optuna framework. For a given set of hyperparameters, the `HyperParameterTUner.objective_fn()` returns the mean validation loss across the K folds, and this mean loss is used as a minimization target by an Optuna `TPESampler` to select new sets of hyperparameters for additional trials. `HyperParameterTuner` also uses an Optuna `MedianPruner` to stop unpromising trials early.


### 6.2 General Approach
The `HyperParameterTuner` class in the `lstm_adversarial_attack.tune_train` sub-package implements a K-fold (default K = 5) cross-validation tuning scheme that utilizes the Optuna framework. When defining the dataset indices for each fold, we oversample from samples in the minority class (label = 1) using a For a given set of hyperparameters, the `HyperParameterTUner.objective_fn()` returns the mean validation loss across the K folds, and this mean loss is used as a minimization target by an Optuna `TPESampler` to select new sets of hyperparameters for additional trials. `HyperParameterTuner` also uses an Optuna `MedianPruner` to stop unpromising trials early.



### 6.2 Check for GPU
Model hyperparameter tuning (along with training, and model attacks) is implemented in PyTorch, and we really need a GPU to run things in a reasonable amount of time.

In [ ]:
import torch

if torch.cuda.is_available():
    cur_device = torch.device("cuda:0")
else:
    cur_device = torch.device("cpu")

print(f"cur_device is {cur_device}")

### 6.3 Instantiate and Examine TunerDriver
We then instantiate a TunerDriver object.

In [ ]:
import lstm_adversarial_attack.tune_train.tuner_driver as td
tuner_driver = td.TunerDriver(
    device=cur_device,
    dataset=dataset,
    continue_study_path=cfg_paths.ONGOING_TUNING_STUDY_PICKLE,
    output_dir=cfg_paths.ONGOING_TUNING_STUDY_DIR,)

The `continue_study_path` and `output_dir` arguments passed to the `TunerDriver` constructor will allow us to build upon an existing Optuna study that contains learning from previously run trials. Model hyperparameter tuning is the most time-consuming part of the overall project pipeline, so we usually do not want to start from scratch. (But if you really want to start from scratch, just don't pass either of these path arguments to the `TuneDriver` constructor). For reference, the values of the two path variables we passed to the TunerDriver constructor are:

In [ ]:
print(f"continue_study_path = {cfg_paths.ONGOING_TUNING_STUDY_PICKLE}")
print(f"output_dir = {cfg_paths.ONGOING_TUNING_STUDY_DIR}")

The TunerDriver object has many default parameters/attributes set by values in `cfg_paths` and `cfg_settings`. Note that its `.tuner` attribute is a `HyperParameterTuner` which in turn has a `.tuning_ranges` attribute that specifies our hyperparameter search space.

In [ ]:
print(f"The tuner_driver.tuner is a {type(tuner_driver.tuner)}\n")
print("tuner_driver.tuner.tuning_ranges:")
pprint.pprint(tuner_driver.tuner.tuning_ranges)

### 6.4 Run the TunerDriver
The code in the next cell will run the Tuner Driver (and its associated HyperParameterTuner). Before starting, a few things to note:
* Depending your GPU compute power, running the full 30 trials could take 2 - 20 hours.
* If the study is stopped early (via CTRL-C or the Jupyter Stop button), learning from whatever trials have completed up to that point will be saved.
* While the tuning trials are running, look ahead to the next Markdown cell for instructions on how to monitor progress in Tensorboard (depending on your notebook output settings you may need to scroll down to see that cell)

In [ ]:
my_completed_study = tuner_driver.run(num_trials=30)

While we are tuning hyperparameters, we can monitor results in Tensorboard. To start the tensorboard server, open a terminal in jupyter lab, and run:
```
$ /bin/bash
$ tensorboard --logdir=/home/devspace/project/data/hyperparameter_tuning/continued_trials/tensorboard --host=0.0.0.0
```
Then, in your browser, go to: `http://localhost:6006/`

You should see something like the screenshot below. Here, we are displaying the full data from trial #20 and are part way through trial #21. We can use the expand arrows on the right to display data from other trials.

![tensorboard_image](images/tensorboard_screenshot_3.png)



## 7. Model Training